In [ ]:
import numpy as np
import graphinglib as gl
import dill
import pyregion

from src.tools.statistics.advanced_stats import increments
from src.hdu.maps.map import Map

In [ ]:
def get_plottables(data: np.ndarray, number_of_bins: int, increment: float) -> list:
    sorted_data = np.sort(np.concatenate((data, -data)))

    bin_width = 0.5
    # bin_width = 125
    bins = np.arange(np.nanmin(sorted_data), np.nanmax(sorted_data) + bin_width*0.99, bin_width)
    max_bin = np.nanmax(sorted_data + bin_width)//bin_width * bin_width
    bins = np.arange(-max_bin, max_bin, bin_width)
    hist_values, hist_edges = np.histogram(sorted_data, bins=bins)

    hist = gl.Histogram(
        sorted_data,
        number_of_bins=bins,
        # number_of_bins=number_of_bins,
        show_params=False
    )
    amplitude = np.max(hist.bin_heights)

    stddev = hist.bin_centers[np.argmin(np.abs(hist_values - np.max(hist_values)/2))] / (np.sqrt(2*np.log(2)))
    # stddev = hist_edges[np.argmin(np.abs(hist_values - np.max(hist_values)/2)) + 1] / (np.sqrt(2*np.log(2)))
    # Half the bin_width could be added so the gaussian passes in the middle of the bins

    curve = gl.Curve.from_function(
        func=lambda x: amplitude * np.exp(-x**2/(2*stddev**2)),
        x_min=hist.bin_edges[0],
        x_max=hist.bin_edges[-1],
        color="black"
    )

    text = gl.Text(
        hist.bin_edges[0]*0.8,
        np.max(hist.bin_heights)*0.8,
        rf"$\Delta={increment}$"
    )

    return [hist, curve]#, text]

### All increments

In [ ]:
turbulence = Map.load("summer_2023/gaussian_fitting/maps/computed_data_selective/turbulence.fits")

regions = [
    ("global_region", None, 40),
    ("diffuse_region", pyregion.open("summer_2023/gaussian_fitting/regions/region_1.reg"), 30),
    ("central_region", pyregion.open("summer_2023/gaussian_fitting/regions/region_2.reg"), 20),
    ("filament_region", pyregion.open("summer_2023/gaussian_fitting/regions/region_3.reg"), 10)
]

In [ ]:
all_figs = []

VAL = 0.5

for name, region, number_of_bins in regions:
    increments_data = increments(turbulence.get_masked_region(region).data)

    figs = []

    for increment in [2,5,10,20]:
        fig = gl.Figure(x_label=" ", y_label=" ")#, title=name)
        data = []
        for increment_i, values in increments_data.items():
            if increment + VAL > increment_i > increment - VAL:
                data.extend(values.tolist())
        hist, curve = get_plottables(np.array(data), number_of_bins, increment)
        fig.add_elements(hist, curve)
        fig.x_lim = hist.bin_edges[0]*0.99, - hist.bin_edges[0]*0.99
        # fig.add_elements(*get_plottables(increments_data[increment], number_of_bins, increment))
        figs.append(fig)
        
    all_figs.extend(figs)

    # multifig = gl.MultiFigure.from_grid(figs, (2,2), (14, 9))
    # multifig.show()
    # multifig.save(f"figures/sh158/advanced_stats/increments/{name}.pdf")

figs_array = np.array(all_figs).reshape((4,4))

figs_array[0,0].title = r"$\Delta=2$"
figs_array[0,1].title = r"$\Delta=5$"
figs_array[0,2].title = r"$\Delta=10$"
figs_array[0,3].title = r"$\Delta=20$"

giga_multifig = gl.MultiFigure.from_grid(figs_array.flatten().tolist(), (4,4), (13.065,9))

giga_multifig.x_label = "Normalized Increment [-]"
giga_multifig.y_label = "Normalized Count [-]"
giga_multifig.title = " "

giga_multifig.save(f"figures/sh158/advanced_stats/increments/increments_article_new.pdf", dpi=300)